## 데이터 검증 프레임워크

- TFDV
    - TFDV(tensorflow-datavalidate)는 거의 텐서플로우 사용이 가능하고, 다른 모델의 사용에 어려움이 있음
    - IDB에서 주로 다루는 데이터 특성에 따라 TFDV의 데이터 기술통계 내용으로는 부족
    - TFX를 KFP로 변환할 수 있어 혼합해서 사용하는 방안을 생각했으나, 데이터 type이 크게 변화해 대용량 데이터 처리에는 비효율적임 -> tensorflow 모델이면 사용하기 좋지만, 다른 모델이면 TFX 사용하는 의미가 크게 없음

- custom
    - TFDV보다 다양한 통계 기능을 제공
    - 대부분의 계산 numpy 기반으로 (일부는 scipy) 제작해 여러 모델을 테스트 할 수 있음
    - Data 통계분석과 검증 결과를 report 형태로 저장해 한눈에 파악
    - KFP 데이터 전처리 파이프라인에 데이터 분석 뿐만 아니라 skew&drift 감지 


참고 : https://github.com/jinwoo1990/mlops-with-tensorflow.git

### Data Variable

raw 데이터

- base_ds : 기존 가지고 있는 데이터
- eval_ds : 새로 들어오는 데이터

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/penguins.csv')

from sklearn.model_selection import train_test_split
base_ds, eval_ds = train_test_split(df, test_size=0.2, random_state=47)

base_ds.index = range(len(base_ds))

base_ds.to_csv('data/base_ds.csv', index=False)
eval_ds.to_csv('data/eval_ds.csv', index=False)

#### Tensor

- 데이터프레임을 하나의 Tensor로 생각해 Tensor 객체로 저장
- Variable 객체는 numpy ndarray로 입력
- 현재는 기술통계 및 array 처리 기능까지 제공 -> 추후 다른 데이터 처리 추가 (fft, 노이즈 처리, 데이터 변환)

In [2]:
# Variable
from variable import Tensor
tensor = Tensor(data= df.to_numpy(), name=df.columns.to_list())
print(tensor)

Tensor([['Adelie' 'Torgersen' 39.1 ... 3750.0 'male' 2007]
          ['Adelie' 'Torgersen' 39.5 ... 3800.0 'female' 2007]
          ['Adelie' 'Torgersen' 40.3 ... 3250.0 'female' 2007]
          ...
          ['Chinstrap' 'Dream' 49.6 ... 3775.0 'male' 2009]
          ['Chinstrap' 'Dream' 50.8 ... 4100.0 'male' 2009]
          ['Chinstrap' 'Dream' 50.2 ... 3775.0 'female' 2009]]),
 name=['species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex', 'year'])


In [3]:
# 데이터 형태
print("row 개수\n",tensor.nrow, end='\n\n')
print("shape\n",tensor.shape, end='\n\n')
print("ndim\n",tensor.ndim, end='\n\n')
print("size\n",tensor.size(), end='\n\n')
print("reshape\n",tensor.reshape(-1,1), end='\n\n')
print("전치행렬\n",tensor.transpose(), end='\n\n')

row 개수
 [344, 344, 344, 344, 344, 344, 344, 344]

shape
 (344, 8)

ndim
 2

size
 2752

reshape
 [['Adelie']
 ['Torgersen']
 [39.1]
 ...
 [3775.0]
 ['female']
 [2009]]

전치행렬
 [['Adelie' 'Adelie' 'Adelie' ... 'Chinstrap' 'Chinstrap' 'Chinstrap']
 ['Torgersen' 'Torgersen' 'Torgersen' ... 'Dream' 'Dream' 'Dream']
 [39.1 39.5 40.3 ... 49.6 50.8 50.2]
 ...
 [3750.0 3800.0 3250.0 ... 3775.0 4100.0 3775.0]
 ['male' 'female' 'female' ... 'male' 'male' 'female']
 [2007 2007 2007 ... 2009 2009 2009]]



In [4]:
# 데이터 타입
print("데이터 타입 (dtype)\n", tensor.dtype, end='\n\n')
print("\n특정 dtype의 인덱싱\n", tensor.dindex('nums'), end='\n\n')
print("\n특정 dtype의 슬라이싱\n", tensor.dslice('str'), end='\n\n')
print("\nastype\n", tensor.dslice('int').astype('str'), end='\n\n')

데이터 타입 (dtype)
 ['str', 'str', 'float', 'float', 'float', 'float', 'str', 'int']


특정 dtype의 인덱싱
 (array([7, 2, 3, 4, 5]), array(['year', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm',
       'body_mass_g'], dtype='<U17'))


특정 dtype의 슬라이싱
 [['Adelie' 'Torgersen' 'male']
 ['Adelie' 'Torgersen' 'female']
 ['Adelie' 'Torgersen' 'female']
 ...
 ['Chinstrap' 'Dream' 'male']
 ['Chinstrap' 'Dream' 'male']
 ['Chinstrap' 'Dream' 'female']]


astype
 ['2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007'
 '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007'
 '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007'
 '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007'
 '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007' '2007'
 '2008' '2008' '2008' '2008' '2008' '2008' '2008' '2008' '2008' '2008'
 '2008' '2008' '2008' '2008' '2008' '2008' '2008' '2008' '2008' '2008'
 '2008' '2008' '2008' '2008' '2008' '2008' '2008

In [5]:
# 데이터 통계
print("sum\n", tensor.sum(axis=0), end='\n\n')
print("mean\n", tensor.mean(axis=0), end='\n\n')
print("std\n", tensor.std(axis=0), end='\n\n')
print("max\n", tensor.max(axis=0), end='\n\n')
print("min\n", tensor.min(axis=0), end='\n\n')
print("quantile\n", tensor.quantile(q=0.5, axis=0), end='\n\n')
print("skew\n", tensor.skew(axis=0), end='\n\n')
print("kurtosis\n", tensor.kurtosis(axis=0), end='\n\n')
print("mode (최대빈도값)\n", Tensor(tensor.dslice('str')).mode(axis=0), end='\n\n')
print("isnull\n", tensor.isnull(), end='\n\n')

sum
 [690762.     nan     nan     nan     nan]

mean
 [2008.02906977   43.92192982   17.15116959  200.91520468 4201.75438596]

std
 [  0.81716559   5.45159602   1.97190392  14.04114057 800.78122924]

max
 [2009.    59.6   21.5  231.  6300. ]

min
 [2007.    32.1   13.1  172.  2700. ]

quantile
 [2008.     44.45   17.3   197.   4050.  ]

skew
 [-0.05349321  0.05288481 -0.14283463  0.34416383  0.46826396]

kurtosis
 [-1.5005427  -0.88076458 -0.91115469 -0.98743443 -0.72624259]

mode (최대빈도값)
 ['Adelie' 'Biscoe' 'male']

isnull
 [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]



In [6]:
# 데이터 분포
print("histogram\n", tensor.histogram(bins=10, density=False), end='\n\n')
print("연속 분포\n", tensor.density(bins=10), end='\n\n')
print("분포의 단위 bucket\n", tensor.bucket(bins=10, smoothing=False), end='\n\n')
print("unique\n", Tensor(tensor.dslice('str')).unique(return_counts=True), end='\n\n')
print("freq (datetime의 주기)\n", tensor.freq(unit='s'), end='\n\n')
print("cycle (datatime의 순환성)\n", tensor.cycle(unit='s'), end='\n\n')

histogram
 [(array([110,   0,   0,   0,   0, 114,   0,   0,   0, 120]), array([2007. , 2007.2, 2007.4, 2007.6, 2007.8, 2008. , 2008.2, 2008.4,
       2008.6, 2008.8, 2009. ])), (array([ 9, 40, 57, 48, 49, 55, 61, 16,  5,  2]), array([32.1 , 34.85, 37.6 , 40.35, 43.1 , 45.85, 48.6 , 51.35, 54.1 ,
       56.85, 59.6 ])), (array([21, 35, 32, 34, 44, 55, 56, 39, 16, 10]), array([13.1 , 13.94, 14.78, 15.62, 16.46, 17.3 , 18.14, 18.98, 19.82,
       20.66, 21.5 ])), (array([ 3, 22, 52, 79, 44, 15, 42, 42, 28, 15]), array([172. , 177.9, 183.8, 189.7, 195.6, 201.5, 207.4, 213.3, 219.2,
       225.1, 231. ])), (array([15, 43, 71, 53, 42, 41, 28, 27, 16,  6]), array([2700., 3060., 3420., 3780., 4140., 4500., 4860., 5220., 5580.,
       5940., 6300.]))]

연속 분포
 [([31.33614327378935, 18.328318329978917, 10.93008649563506, 18.926879831276178, 32.47415104407778, 32.481650421611405, 19.01042994300341, 11.418036499279285, 19.907928502188568, 34.17668341150907], [2007.0, 2007.2, 2007.4, 2007.6000000000

### Data Validation

#### DataInfer
데이터프레임 추론 후 stat report 생성

In [7]:
from validation import DataInfer
stat = DataInfer(data=df, bins=5, smoothing=False, unit_of_time='s')

In [8]:
# schema : field, n_row, n_null, dtype
stat.infer_schema()

schema(field=['species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex', 'year'], n_row=[344, 344, 344, 344, 344, 344, 344, 344], n_null=array([ 0,  0,  2,  2,  2,  2, 11,  0]), dtype=['str', 'str', 'float', 'float', 'float', 'float', 'str', 'int'])

In [9]:
# nums : 평균, 분산, quantile range, 왜도, 첨도, buckets
# bucket에는 (low 값, high 값, 개수) 형태로 저장
stat.infer_num()

num_stat(field=['year', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], mean=array([2008.02906977,   43.92192982,   17.15116959,  200.91520468,
       4201.75438596]), std=array([  0.81716559,   5.45159602,   1.97190392,  14.04114057,
       800.78122924]), min=array([2007. ,   32.1,   13.1,  172. , 2700. ]), q1=array([2007.   ,   39.225,   15.6  ,  190.   , 3550.   ]), median=array([2008.  ,   44.45,   17.3 ,  197.  , 4050.  ]), q3=array([2009. ,   48.5,   18.7,  213. , 4750. ]), max=array([2009. ,   59.6,   21.5,  231. , 6300. ]), mode=masked_array(data=[2009. ,   41.1,   17. ,  190. , 3800. ],
             mask=False,
       fill_value=1e+20), skew=array([-0.05349321,  0.05288481, -0.14283463,  0.34416383,  0.46826396]), kurtosis=array([-1.5005427 , -0.88076458, -0.91115469, -0.98743443, -0.72624259]), bucket=[[(2007.0, 2007.4, 110), (2007.4, 2007.8, 0), (2007.8, 2008.2, 114), (2008.2, 2008.6, 0), (2008.6, 2009.0, 120)], [(32.1, 37.6, 49), (37.6, 43.1, 105), 

In [10]:
# category : unique, counts
stat.infer_cat()

cat_stat(field=['species', 'island', 'sex'], lbs=[['Adelie', 'Chinstrap', 'Gentoo'], ['Biscoe', 'Dream', 'Torgersen'], ['female', 'male', 'nan']], cnt=[[152, 68, 124], [168, 124, 52], [165, 168, 11]])

In [11]:
# datatime : freq, cycle
stat.infer_dt()

datetime(field=[], start=[], end=[], freq=[], cycle=[])

In [12]:
# data stats
stat_result = stat.build()
stat_result

{'schema': {'field': ['species',
   'island',
   'bill_length_mm',
   'bill_depth_mm',
   'flipper_length_mm',
   'body_mass_g',
   'sex',
   'year'],
  'n_row': [344, 344, 344, 344, 344, 344, 344, 344],
  'n_null': [0, 0, 2, 2, 2, 2, 11, 0],
  'dtype': ['str', 'str', 'float', 'float', 'float', 'float', 'str', 'int']},
 'num_stat': {'field': ['year',
   'bill_length_mm',
   'bill_depth_mm',
   'flipper_length_mm',
   'body_mass_g'],
  'mean': [2008.0290697674418,
   43.921929824561424,
   17.15116959064328,
   200.91520467836258,
   4201.754385964912],
  'std': [0.8171655889620321,
   5.45159602316182,
   1.9719039187562528,
   14.041140568589102,
   800.7812292384524],
  'min': [2007.0, 32.1, 13.1, 172.0, 2700.0],
  'q1': [2007.0, 39.225, 15.6, 190.0, 3550.0],
  'median': [2008.0, 44.45, 17.3, 197.0, 4050.0],
  'q3': [2009.0, 48.5, 18.7, 213.0, 4750.0],
  'max': [2009.0, 59.6, 21.5, 231.0, 6300.0],
  'mode': [2009.0, 41.1, 17.0, 190.0, 3800.0],
  'skew': [-0.053493206350346406,
   0.0

#### DataValidate
base에 대한 eval stat 검증

In [13]:
from validation import DataValidate
dv = DataValidate()

# 임계치 지정
dv.jsd_threshold = 0.001
dv.lin_threshold = 0.001

In [14]:
base_stat = DataInfer(data=base_ds, bins=10, smoothing=False, unit_of_time='s').build()
eval_stat = DataInfer(data=eval_ds, bins=10, smoothing=False, unit_of_time='s').build()

연속형 데이터 : jesen-shannon divergence
- kld : kullback-liebler divergence, 두 확률분포의 차이를 계산
- jsd : jensen-shannon divergence, 두 확률분포의 유사성을 계산
- $JSD(p||q) = \frac{1}{2}KLD(p||m) + \frac{1}{2}KLD(q||m),$ $where$ $m = \frac{1}{2}(p+q)$
- 연속형 변수의 검증 척도
- base의 히스토그램을 기준으로 eval 데이터가 얼마나 유사한지 검증

In [15]:
num_list = eval_stat['num_stat']['field']
dv.jensen_shannon_divergence(base=base_stat, eval=eval_stat, field=num_list[0])

0.006354680473682012

범주형 데이터 : L-infinite norm
- $||x||_{inf} = max|x_n|$
- 범주형 변수의 검증 척도
- base의 범주형 정보를 기준으로 eval 데이터의 거리를 검증

In [16]:
cat_list = eval_stat['cat_stat']['field']
dv.l_inf_norm(base=base_stat, eval=eval_stat, field=cat_list[0])

0.06598155467720684

In [17]:
p = np.arange(0.0, 10.0)
q = np.arange(1.0, 11.0)

In [18]:
idx = np.where((p != 0)&(q != 0))[0]
p = p[idx]
q = q[idx]
p * np.log(p/q)

array([-0.69314718, -0.81093022, -0.86304622, -0.89257421, -0.91160778,
       -0.92490408, -0.93471975, -0.94226429, -0.94824464])

In [19]:
# 전체 distance
dv.distance(base=base_stat, eval = eval_stat)

{'num_stat': {'flipper_length_mm': 0.02266450206044484,
  'body_mass_g': 0.061875195466568514,
  'year': 0.006354680473682012,
  'bill_depth_mm': 0.03584001921579724,
  'bill_length_mm': 0.02387054348279568},
 'cat_stat': {'species': 0.06598155467720684,
  'island': 0.10645586297760212,
  'sex': 0.14217536071032189}}

In [20]:
# descripts
dv.descripts

{'field': 'anoamly fields between base data and eval data.',
 'dtype': 'anoamly data types of fields between base data and eval data.',
 'n_null': 'absolute difference of null ratio (n_null/n_row) between base data and eval data.',
 'num_stat': {'distance': 'anomaly jensen-shannon distance over threshold 0.01.',
  'skew': 'anomaly skewness over threshold 0.5.',
  'kurtosis': 'anomaly kurtosis over threshold 0.5.'},
 'cat_stat': {'distance': 'anomaly jensen-shannon distance over threshold 0.01.',
  'unique': 'anomaly unique labels between base data and eval data.',
  'count': 'anomaly count ratio (base connts/eval counts) each label.'},
 'threshold': {'jsd_threshold': 'threshold of jensen-shannon divergence. If exceeded, determined as anomalies.',
  'lin_threshold': 'threshold of L-infinite norm. If exceeded, determined as anomalies.',
  'num_stat_treshold': 'threshold of numerical stat (skewness, kurotisis) difference between base and eval. If exceeded, determined as anomalies.'}}

In [21]:
# report 생성
report = dv.build(base_stat, eval_stat)

In [22]:
from validation import Converter
cv = Converter()
print(cv.to_json(report))

{
    "field": null,
    "ftype": null,
    "n_null": {
        "bill_length_mm": 0.007272727272727273,
        "bill_depth_mm": 0.007272727272727273,
        "flipper_length_mm": 0.007272727272727273,
        "body_mass_g": 0.007272727272727273,
        "sex": 0.021870882740447954
    },
    "num_stat": {
        "distance": {
            "flipper_length_mm": 0.02266450206044484,
            "body_mass_g": 0.061875195466568514,
            "year": 0.006354680473682012,
            "bill_depth_mm": 0.03584001921579724,
            "bill_length_mm": 0.02387054348279568
        },
        "skew": {},
        "kurtosis": {}
    },
    "cat_stat": {
        "distance": {
            "species": 0.06598155467720684,
            "island": 0.10645586297760212,
            "sex": 0.14217536071032189
        },
        "lbs": {},
        "cnt": {
            "species": {
                "Adelie": 0.26666666666666666,
                "Chinstrap": 0.1724137931034483,
                "Gentoo": 0.27

In [23]:
cv.save(stat_result, 'stat.json')
cv.save(report, 'report.json')

In [24]:
import tensorflow_data_validation as tfdv
tfdv_base_stat = tfdv.generate_statistics_from_dataframe(base_ds)
tfdv_eval_stat = tfdv.generate_statistics_from_dataframe(eval_ds)
tfdv_schema = tfdv.infer_schema(tfdv_base_stat)

dataset_idx = 0
type_dict = {0: 'INT', 1: 'FLOAT', 2: 'STRING', 3: 'BYTES', 4: 'STRUCT'}
train_datasets = tfdv_base_stat.datasets[dataset_idx]

feat_dict = {'INT': {}, 'FLOAT': {}, 'STRING': {}, 'BYTES': {}, 'STRUCT': {}}
for i in range(len(train_datasets.features)):
    feat_type = type_dict[train_datasets.features[i].type]
    feat_name = train_datasets.features[i].path.step[0]
    feat_dict[feat_type][i] = feat_name

for item in feat_dict['STRING'].values():
    tfdv.get_feature(tfdv_schema, item).skew_comparator.infinity_norm.threshold = 0.0

for item in feat_dict['INT'].values():
    tfdv.get_feature(tfdv_schema, item).skew_comparator.jensen_shannon_divergence.threshold = 0.0

skew_anomalies = tfdv.validate_statistics(tfdv_base_stat, tfdv_schema, serving_statistics=tfdv_eval_stat)
tfdv.display_anomalies(skew_anomalies)

2023-06-29 15:37:44.550904: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-29 15:37:44.592160: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 15:37:45.548096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


,Anomaly short description,Anomaly long description
Feature name,,
'island',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.106456 (up to six significant digits), above the threshold 0. The feature value with maximum difference is: Dream"
'year',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.00918602 (up to six significant digits), above the threshold 0."
'sex',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.142175 (up to six significant digits), above the threshold 0. The feature value with maximum difference is: male"
'species',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.0659816 (up to six significant digits), above the threshold 0. The feature value with maximum difference is: Chinstrap"


In [25]:
tfdv_result = {}
for k in range(len(tfdv_base_stat.datasets[0].features)):
    try:
        feat_name = skew_anomalies.drift_skew_info[k].path.step[0]
        skew_value = skew_anomalies.drift_skew_info[k].skew_measurements[0].value
        tfdv_result[feat_name] = skew_value
    except:
        pass

custom_result = {}
for k, v in dv.distance(base=base_stat, eval = eval_stat).items():
    custom_result.update(v)

In [26]:
tfdv_result

{'island': 0.10645586297760212,
 'sex': 0.14217536071032189,
 'species': 0.06598155467720684,
 'year': 0.00918601913353076}

In [27]:
dict(sorted(custom_result.items()))

{'bill_depth_mm': 0.03584001921579724,
 'bill_length_mm': 0.02387054348279568,
 'body_mass_g': 0.061875195466568514,
 'flipper_length_mm': 0.02266450206044484,
 'island': 0.10645586297760212,
 'sex': 0.14217536071032189,
 'species': 0.06598155467720684,
 'year': 0.006354680473682012}

### TFDV

In [28]:
import tfx.v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_data_validation as tfdv

context = InteractiveContext()

In [29]:
from  tfx.proto import example_gen_pb2
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen

input = example_gen_pb2.Input(splits=[
                example_gen_pb2.Input.Split(name='train', pattern='base_ds.csv'),
                example_gen_pb2.Input.Split(name='eval', pattern='eval_ds.csv')
            ])
example_gen = CsvExampleGen(input_base='data', input_config=input)
context.run(example_gen, enable_cache=True)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [30]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2023-06-29T15_37_49.180376-0mgyl6_6/CsvExampleGen/examples/1


In [31]:
import os
import tensorflow as tf

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# iterate first 5 records and decode \
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)

2023-06-29 15:37:52.089314: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.


features {
  feature {
    key: "bill_depth_mm"
    value {
      float_list {
        value: 18.5
      }
    }
  }
  feature {
    key: "bill_length_mm"
    value {
      float_list {
        value: 42.79999923706055
      }
    }
  }
  feature {
    key: "body_mass_g"
    value {
      float_list {
        value: 4250.0
      }
    }
  }
  feature {
    key: "flipper_length_mm"
    value {
      float_list {
        value: 195.0
      }
    }
  }
  feature {
    key: "island"
    value {
      bytes_list {
        value: "Torgersen"
      }
    }
  }
  feature {
    key: "sex"
    value {
      bytes_list {
        value: "male"
      }
    }
  }
  feature {
    key: "species"
    value {
      bytes_list {
        value: "Adelie"
      }
    }
  }
  feature {
    key: "year"
    value {
      int64_list {
        value: 2008
      }
    }
  }
}

features {
  feature {
    key: "bill_depth_mm"
    value {
      float_list {
        value: 13.699999809265137
      }
    }
  }
  featu

2023-06-29 15:37:52.112096: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-29 15:37:52.112574: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-29 15:37:52.114630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-29 15:37:52.115182: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-29 15:37:52.115675: I tensorflow/compile

In [32]:
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [33]:
context.show(statistics_gen.outputs['statistics'])

In [34]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [35]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'bill_depth_mm',FLOAT,required,,-
'bill_length_mm',FLOAT,required,,-
'body_mass_g',FLOAT,required,,-
'flipper_length_mm',FLOAT,required,,-
'island',STRING,required,single,'island'
'sex',STRING,required,,'sex'
'species',STRING,required,single,'species'
'year',INT,required,single,-


,Values
Domain,
'island',"'Biscoe', 'Dream', 'Torgersen'"
'sex',"'female', 'male'"
'species',"'Adelie', 'Chinstrap', 'Gentoo'"


In [36]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'],
    )
context.run(example_validator, enable_cache=True)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [37]:
context.show(example_validator.outputs['anomalies'])